In [ ]:
from pylab import *

import skimage

import numpy as np

In [ ]:
def lpfilter(filt_type, Ny, Nx, sigma, n=1):
    
    if (Ny%2 == 0):
        y = np.arange(0,Ny) - Ny/2 + 0.5
    else:
        y = np.arange(0,Ny) - (Ny-1)/2
    
    if (Nx%2 == 0):
        x = np.arange(0,Nx) - Nx/2 + 0.5
    else:
        x = np.arange(0,Nx) - (Nx-1)/2

    
    X, Y = meshgrid(x, y)
    
    D = np.sqrt(np.square(X) + np.square(Y))
    
    if filt_type == 'gaussian':
        filter_mask = exp(-np.square(D)/(2*np.square(sigma)))
    elif filt_type == 'btw':
        filter_mask = 1/(1+(D/sigma)**(2*n))
    elif filt_type == 'ideal':
        filter_mask = ones([Ny, Nx])
        filter_mask[D>sigma] = 0
    else:
        print('Greška! Nije podržan tip filtra: ', filt_type)
        return
    
    return filter_mask

In [ ]:
print(lpfilter('btw', 5, 5, 1))

In [ ]:
img = skimage.img_as_float(imread('sekvence/square.tif'))

[Ny, Nx] = shape(img)

img_fft = fftshift(fft2(img))
filt_freq = lpfilter('gaussian', Ny, Nx, sigma=10)

img_fft_filt = img_fft*filt_freq

img_filt = real(ifft2(ifftshift(img_fft_filt)))
img_filt = np.clip(img_filt, 0, 1)

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16,8), dpi=120)
ax = axes.ravel()

ax[0].imshow(img, cmap='gray'); ax[0].set_axis_off(); ax[0].set_title('Ulazna slika')
ax[1].imshow(img_filt, cmap='gray'); ax[1].set_axis_off(); ax[1].set_title('Filtrirana slika');

In [ ]:
img = skimage.img_as_float(imread('sekvence/square.tif'))
[Ny, Nx] = shape(img)

Px = 2*Nx-1; Py = 2*Ny-1

img_pad = zeros([Py,Px])
img_pad[0:Ny, 0:Nx] = img

img_pad_fft = fftshift(fft2(img_pad))
filt_freq = lpfilter('gaussian', Py, Px, sigma=20)

img_pad_fft_filt = img_pad_fft*filt_freq

img_pad_filt = real(ifft2(ifftshift(img_pad_fft_filt)))
img_filt = np.clip(img_pad_filt[0:Ny, 0:Nx], 0, 1)

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16,8), dpi=120)
ax = axes.ravel()

ax[0].imshow(img, cmap='gray'); ax[0].set_axis_off(); ax[0].set_title('Ulazna slika')
ax[1].imshow(img_filt, cmap='gray'); ax[1].set_axis_off(); ax[1].set_title('Filtrirana slika');

In [ ]:
h_lap = [[1,  1, 1],
         [1, -8, 1],
         [1,  1, 1]]

h_lap_p = zeros([64, 64])
h_lap_p[0:3,0:3] = h_lap

H_lap_p = fftshift(fft2(h_lap_p))

x = list(range(0,64))
y = list(range(0,64))

X, Y = meshgrid(x, y)

fig = plt.figure(figsize=(8,8), dpi=120)
ax = plt.axes(projection='3d')
ax.plot_surface(X, Y, abs(H_lap_p), cmap=cm.coolwarm); ax.set_title('Laplasijan');

In [ ]:
h_avg = ones([9,9])/81

h_avg_p = zeros([64, 64])
h_avg_p[0:9,0:9] = h_avg

H_avg_p = fftshift(fft2(h_avg_p))

x = list(range(0,64))
y = list(range(0,64))

X, Y = meshgrid(x, y)

fig = plt.figure(figsize=(8,8), dpi=120)
ax = plt.axes(projection='3d')
ax.plot_surface(X, Y, abs(H_avg_p), cmap=cm.coolwarm); ax.set_title('Usrednjavanje 9x9');

In [ ]:
xg = list(range(-7,8))
yg = list(range(-7,8))

Xg, Yg = meshgrid(xg, yg)

sigma = 2

h_gaus = (1/(2*pi*(sigma**2)))*exp(-(np.square(Xg) + np.square(Yg))/(2*(sigma**2)))

h_gaus_p = zeros([64, 64])
h_gaus_p[0:15,0:15] = h_gaus

H_gaus_p = fftshift(fft2(h_gaus_p))

x = list(range(0,64))
y = list(range(0,64))

X, Y = meshgrid(x, y)

fig = plt.figure(figsize=(8,8), dpi=120)
ax = plt.axes(projection='3d')
ax.plot_surface(X, Y, abs(H_gaus_p), cmap=cm.coolwarm); ax.set_title('Gausovo usrednjavanje');

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
def plot_filters(filters):
    N = len(filters)
    
    fig = plt.figure(figsize=(18, 6*N), dpi=120)
    
    for i in range(0,N):
        ax1 = fig.add_subplot(N, 3, 3*i + 1, projection='3d')
        ax2 = fig.add_subplot(N, 3, 3*i + 2, projection='3d')
        ax3 = fig.add_subplot(N, 3, 3*i + 3)
        
        filter_ifft = real(ifftshift(ifft2(filters[i])))
        
        x = list(range(0,101))
        y = list(range(0,101))

        X, Y = meshgrid(x, y)

        ax1.plot_surface(X, Y, abs(filters[i]), cmap='coolwarm');
        ax2.plot_surface(X, Y, abs(filter_ifft), cmap='coolwarm');
        ax3.plot(x, abs(filter_ifft[51,:]));                       

    ax = fig.axes 
    ax[0].set_title('Frekvencijski domen')
    ax[1].set_title('Prostorni domen')
    ax[2].set_title('Presek');

In [ ]:
lp_ideal = list()
lp_ideal.append(lpfilter('ideal', 101, 101, 5))
lp_ideal.append(lpfilter('ideal', 101, 101, 10))
lp_ideal.append(lpfilter('ideal', 101, 101, 20))

plot_filters(lp_ideal)

In [ ]:
lp_btw = list()
lp_btw.append(lpfilter('btw', 101, 101, 10))
lp_btw.append(lpfilter('btw', 101, 101, 10, 2))
lp_btw.append(lpfilter('btw', 101, 101, 10, 10))

plot_filters(lp_btw)

In [ ]:
lp_gaus = list()
lp_gaus.append(lpfilter('gaussian', 101, 101, 5))
lp_gaus.append(lpfilter('gaussian', 101, 101, 10))
lp_gaus.append(lpfilter('gaussian', 101, 101, 20))

plot_filters(lp_gaus)

In [ ]:
lp_all = list()
lp_all.append(lpfilter('ideal', 101, 101, 10))
lp_all.append(lpfilter('gaussian', 101, 101, 10))
lp_all.append(lpfilter('btw', 101, 101, 10))
lp_all.append(lpfilter('btw', 101, 101, 10, 3))

plot_filters(lp_all)

In [ ]:
img = skimage.img_as_float(imread('sekvence/lena.tif'))
[Ny, Nx] = shape(img)
Px = 2*Nx-1; Py = 2*Ny-1

img_pad = zeros([Py, Px])
img_pad[0:Ny, 0:Nx] = img

img_pad_fft = fftshift(fft2(img_pad))
lp_filt_freq = lpfilter('gaussian', Py, Px, 40)

img_pad_fft_filt = img_pad_fft*lp_filt_freq

img_pad_filt = real(ifft2(ifftshift(img_pad_fft_filt)))
img_filt = np.clip(img_pad_filt[0:Ny, 0:Nx], 0, 1)

fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(12,8), dpi=120)
ax = axes.ravel()

ax[0].imshow(log(1+abs(img_pad_fft)), cmap='gray'); ax[0].set_axis_off(); ax[0].set_title('Ulazna slika i spektar')
ax[1].imshow(log(1+abs(lp_filt_freq)), cmap='gray'); ax[1].set_axis_off(); ax[1].set_title('Gausov filtar')
ax[2].imshow(log(1+abs(img_pad_fft_filt)), cmap='gray'); ax[2].set_axis_off(); ax[2].set_title('Izlazna slika i spektar')
ax[3].imshow(img, cmap='gray'); ax[3].set_axis_off()
ax[4].set_axis_off()
ax[5].imshow(img_filt, cmap='gray'); ax[5].set_axis_off()

In [ ]:
hp_filt = 1 - lpfilter('ideal', 101, 101, 10)

plot_filters([hp_filt])

In [ ]:
img = skimage.img_as_float(imread('sekvence/lena.tif'))
[Ny, Nx] = shape(img)
Px = 2*Nx-1; Py = 2*Ny-1

img_pad = zeros([Py, Px])
img_pad[0:Ny, 0:Nx] = img

img_pad_ftt = fftshift(fft2(img_pad))
hp_filt_freq = 1 - lpfilter('ideal', Py, Px, 100)

img_pad_fft_filt = img_pad_fft*hp_filt_freq

img_pad_filt = real(ifft2(ifftshift(img_pad_fft_filt)))
img_filt = np.clip(img_pad_filt[0:Ny, 0:Nx], 0, 1)

fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(12,8), dpi=120)
ax = axes.ravel()

ax[0].imshow(log(1+abs(img_pad_fft)), cmap='gray'); ax[0].set_axis_off(); ax[0].set_title('Ulazna slika i spektar')
ax[1].imshow(log(1+abs(hp_filt_freq)), cmap='gray'); ax[1].set_axis_off(); ax[1].set_title('Gausov filtar')
ax[2].imshow(log(1+abs(img_pad_fft_filt)), cmap='gray'); ax[2].set_axis_off(); ax[2].set_title('Izlazna slika i spektar')
ax[3].imshow(img, cmap='gray'); ax[3].set_axis_off()
ax[4].set_axis_off()
ax[5].imshow(img_filt, cmap='gray'); ax[5].set_axis_off()

In [ ]:
img = skimage.img_as_float(imread('sekvence/lena.tif'))
[Ny, Nx] = shape(img)
Py = 2*Ny-1; Px = 2*Nx-1

img_pad = zeros([Py, Px])
img_pad[0:Ny, 0:Nx] = img

img_pad_fft = fftshift(fft2(img_pad))
sharpen_filt_freq = 1 + 1.2*(1 - lpfilter('gaussian', Py, Px, 100))

img_pad_fft_filt = img_pad_fft*sharpen_filt_freq

img_pad_filt = real(ifft2(ifftshift(img_pad_fft_filt)))
img_filt = np.clip(img_pad_filt[0:Ny, 0:Nx], 0, 1)

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16,8), dpi=120)
ax = axes.ravel()

ax[0].imshow(img, cmap='gray'); ax[0].set_axis_off(); ax[0].set_title('Ulazna slika')
ax[1].imshow(img_filt, cmap='gray'); ax[1].set_axis_off(); ax[1].set_title('Izoštrena slika');

In [ ]:
from skimage import exposure

img = skimage.img_as_float(imread('sekvence/appolo17.tif'))

[Ny, Nx] = shape(img)

img_fft = fftshift(fft2(img))

lp1_filter_freq = lpfilter('ideal', Ny, Nx, 101)
lp2_filter_freq = lpfilter('ideal', Ny, Nx, 99)

br_filter_freq = 1 - lp1_filter_freq + lp2_filter_freq

img_fft_filt = img_fft*br_filter_freq

img_filt = real(ifft2(ifftshift(img_fft_filt)))

img_filt = np.clip(img_filt, 0, 1)

img_filt_enhanced = exposure.rescale_intensity(img_filt)

fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(14,12), dpi=120)
ax = axes.ravel()

ax[0].imshow(img, cmap='gray'); ax[0].set_axis_off(); ax[0].set_title('Ulazna slika')
ax[1].imshow(log(1+abs(img_fft)), cmap='gray'); ax[1].set_axis_off(); ax[1].set_title('Spektar ulazne slike')
ax[2].imshow(log(1+abs(br_filter_freq)), cmap='gray'); ax[2].set_axis_off(); ax[2].set_title('Filtar')
ax[3].imshow(log(1+abs(img_fft_filt)), cmap='gray'); ax[3].set_axis_off(); ax[3].set_title('Spektar filtrirane slike')
ax[4].imshow(img_filt, cmap='gray', vmin=0, vmax=1); ax[4].set_axis_off(); ax[4].set_title('Izlaz nakon filtriranja')
ax[5].imshow(img_filt_enhanced, cmap='gray', vmin=0, vmax=1); ax[5].set_axis_off(); ax[5].set_title('Izlaz nakon popravke kontrasta');

In [ ]:
def cnotch(filt_type, notch, Ny, Nx, C, r, n=1):
    N_filters = len(C)
    
    filter_mask = zeros([Ny,Nx])
    
    if (Ny%2 == 0):
        y = np.arange(0,Ny) - Ny/2 + 0.5
    else:
        y = np.arange(0,Ny) - (Ny-1)/2
    
    if (Nx%2 == 0):
        x = np.arange(0,Nx) - Nx/2 + 0.5
    else:
        x = np.arange(0,Nx) - (Nx-1)/2

    X, Y = meshgrid(x, y)
    
    for i in range(0, N_filters):
        C_current = C[i]
        
        C_complement = zeros([2,1])
        C_complement[0] = -C_current[0]
        C_complement[1] = -C_current[1]
        
        if (Ny%2 == 0):
            y0 = y - C_current[0] + Ny/2 - 0.5
        else:
            y0 = y - C_current[0] + (Ny-1)/2
        
        if (Nx%2 == 0):
            x0 = x - C_current[1] + Nx/2 - 0.5
        else:
            x0 = x - C_current[1] + (Nx-1)/2
        
        X0, Y0 = meshgrid(x0, y0)
        
        D0 = np.sqrt(np.square(X0) + np.square(Y0))
    
        if (Ny%2 == 0):
            y0c = y - C_complement[0] - Ny/2 + 0.5
        else:
            y0c = y - C_complement[0] - (Ny-1)/2
        
        if (Nx%2 == 0):
            x0c = x - C_complement[1] - Nx/2 + 0.5
        else:
            x0c = x - C_complement[1] - (Nx-1)/2
        
        X0c, Y0c = meshgrid(x0c, y0c)
        
        D0c = np.sqrt(np.square(X0c) + np.square(Y0c))
    
        if filt_type == 'gaussian':
            filter_mask = filter_mask + \
                          exp(-np.square(D0)/(2*np.square(r))) + \
                          exp(-np.square(D0c)/(2*np.square(r)))
        elif filt_type == 'btw':
            filter_mask = filter_mask + \
                          1/(1+(D0/r)**(2*n)) + \
                          1/(1+(D0c/r)**(2*n))
        elif filt_type == 'ideal':
            filter_mask[(D0<=r)|(D0c<=r)] = 1
        else:
            print('Greška! Nije podržan tip filtra: ', filt_type)
            return
    
    if notch == 'pass':
        return filter_mask
    elif notch == 'reject':
        return 1 - filter_mask
    else:
        return

In [ ]:
from skimage import color

img = color.rgb2gray(skimage.img_as_float(imread('sekvence/tiger_ht.jpg')))

[Ny, Nx] = shape(img)

img_fft = fftshift(fft2(img))

C = [[112, 20], [68, 66], [24, 111], [79, 147]]

nr_filter_freq = cnotch('gaussian', 'reject', Ny, Nx, C, 20)

img_fft_filt = img_fft*nr_filter_freq

img_filt = real(ifft2(ifftshift(img_fft_filt)))
img_filt = np.clip(img_filt, 0, 1)

fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(10,12), dpi=120)
ax = axes.ravel()

ax[0].imshow(img, cmap='gray'); ax[0].set_axis_off(); ax[0].set_title('Ulazna slika')
ax[1].imshow(log(1+abs(img_fft)), cmap='gray'); ax[1].set_axis_off(); ax[1].set_title('Spektar ulazne slike')
ax[2].imshow(log(1+abs(nr_filter_freq)), cmap='gray'); ax[2].set_axis_off(); ax[2].set_title('Filtar')
ax[3].imshow(log(1+abs(img_fft_filt)), cmap='gray'); ax[3].set_axis_off(); ax[3].set_title('Spektar filtrirane slike')
ax[4].imshow(img_filt, cmap='gray', vmin=0, vmax=1); ax[4].set_axis_off(); ax[4].set_title('Izlaz nakon filtriranja')
ax[5].set_axis_off();